In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime, timedelta
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import requests
from openpyxl import *

# 시작 날짜와 종료 날짜 정의
start_date = datetime(2024, 8, 20)
end_date = datetime(2024, 8, 21)

# 날짜 리스트 생성
date_list = [(start_date + timedelta(days=x)).strftime('%Y%m%d') for x in range((end_date - start_date).days + 1)]

for date in tqdm(date_list):
    # 첫 번째 코드에서 지정한 뉴스의 링크들이 담긴 파일
    link = pd.read_excel(f'./crawl/news_{date}.xlsx')
    # 엑셀 파일의 빈칸이 있기에 최종 결과파일을 별도로 생성
    excel_name = f'./newsdata/news_detail_{date}.xlsx'
    Main_link = list(link['link'])

    # number: 기사 순서, title: 기사 제목, information: 본문 내용, link: 기사의 링크
    Information = pd.DataFrame({'number': [], 'title': [], 'information': [], 'link': []})
    # 본문 내용을 추가하기 전이기 때문에 미리 나머지 내용을 담아둠
    Information['number'] = link['number']
    Information['title'] = link['title']
    Information['link'] = link['link']
    information = []

    for main_link in Main_link:
        # 기사가 전체적으로 2개의 구조를 가지고 있음 (게임/리뷰 카테고리에 한하여)
        # 하나의 구조를 기준으로 삼고, 해당 부분에서 오류가 발생하면 다음 구조의 기사로 판단
        try:
            response = requests.get(main_link, headers={"User-Agent": "Mozilla/5.0"})
            if response.status_code == 200:
                html = response.content
                soup = BeautifulSoup(html, 'html.parser')
                # 기사의 본문 내용만 담고 있는 부분
                info = soup.find('div', {'id': 'newsct_article'}).text.strip()
                # 기사 내용 데이터 분석을 위해 줄바꿈을 띄어쓰기로 변경
                info = info.replace('\n', ' ')
                information.append(info)
        except:
            # 다른 구조에서 재 크롤링 코드
            # 여기서 오류가 나는 경우는 게임/리뷰 기사가 아닌 다른 카테고리의 기사로 판단
            try:
                response = requests.get(main_link, headers={"User-Agent": "Mozilla/5.0"})
                if response.status_code == 200:
                    html = response.content
                    soup = BeautifulSoup(html, 'html.parser')
                    # 기사의 본문 내용을 담고 있는 부분
                    info = soup.find('div', {'id': 'newsEndContents'}).text.strip()
                    info = info.replace('\n', ' ')
                    # 해당 구조의 기사는 기자의 정보가 본문과 무조건 같이 존재
                    # 기자의 정보 부분은 필요가 없기 때문에 기자 정보의 기준점이 되는 부분을 찾음
                    # 기자의 정보 기준이 기재정보라는 단어이기 때문에 그 이후는 삭제
                    '''
                    end = info.index('기재정보')
                    info = info[:end]
                    '''
                    information.append(info)
            except Exception as e:
                info = ''
                information.append(info)
                # 오류가 발생하는 이유와 발생하는 링크를 출력하여 오류를 확인하는 장치
                # print(e)
                # print(main_link)

    Information['information'] = information
    
    Information.to_csv(f'./newsdata/news_detail_{date}.csv', encoding='cp949',index=False)
    print(f'엑셀 파일 저장 완료: {excel_name}')
 
    '''with pd.ExcelWriter(excel_name) as writer:
        Information.to_excel(writer, sheet_name='결과값', index=False)
        print(f'엑셀 파일 저장 완료: {excel_name}')'''

  0%|          | 0/2 [00:00<?, ?it/s]

UnicodeEncodeError: 'cp949' codec can't encode character '\xa0' in position 481: illegal multibyte sequence

8월 20일 계속 오류남